### Création du jeu ###

dans un premier temps on créer une class TicTacToe qui represente l'environement du jeu

In [2]:
import random
import numpy as np
from jupyter_server.terminal import initialize


class TicTacToeEnv:
    def __init__(self):
        self.board = [0] * 9  # 9 positions sur le plateau
        self.current_winner = None

    #reset les case du plateau
    def reset(self):
        self.board = [0] * 9
        self.current_winner = None
        return tuple(self.board)

    #indique les case encore idsponible pour jouer son tour
    def available_moves(self):
        return [i for i, x in enumerate(self.board) if x == 0]

    #pose un pion sur le plateau, renvois false si la case est deja prise et renvois true si le coup fait ganger le joueur
    def make_move(self, position, player):
        if self.board[position] == 0:
            self.board[position] = player
            if self.check_winner(position, player):
                self.current_winner = player
            return True
        return False

    #liste des combinaison gagnants
    def check_winner(self, square, player):
        # Conditions de victoire
        win_conditions = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # lignes
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # colonnes
            [0, 4, 8], [2, 4, 6]  # diagonales
        ]
        for condition in win_conditions:
            if square in condition and all(self.board[i] == player for i in condition):
                return True
        return False

    #indique si la partie est une egalité
    def is_draw(self):
        return 0 not in self.board and self.current_winner is None
    
    #renvois l'etiat actuel du tableau, used dasn le stockage de la Q-table
    def get_state(self):
        return tuple(self.board)


    #affichage du plateau
    def render(self):
        # Affichage du plateau
        for row in [self.board[i:i + 3] for i in range(0, 9, 3)]:
            print(" | ".join([str(x) if x != 0 else " " for x in row]))
            print("-" * 5)


C:\Users\guill\AppData\Local\Temp\ipykernel_68016\3454935457.py:3: DeprecationWarning: Terminals support has moved to `jupyter_server_terminals`
  from jupyter_server.terminal import initialize


### Classe de training Ai vs Random ###

Dans un premier temps on entraine chaque ia contre un joueur qui joue aléatoirement pour qu'elle recupére un niveau minimum au jeu

cette action nous permet d'etre plus efficace sur le training Ai contre Ai

In [3]:
def trainAiVsRandom(ai_agent, env, episodes, ai_starts):
    
    #on définie un joueur qui va jouer aléatoirement a chaque tour
    def random_player_move(available_moves):
        return random.choice(available_moves)

    win, loss, draw = 0, 0, 0  # Compteurs pour les statistiques de jeu

    for episode in range(1, episodes + 1):
        state = env.reset()  # Réinitialise l'environnement pour chaque partie
        done = False
        turn = 1 if ai_starts else -1  # Si ai_starts est True, l'IA commence ; sinon, le joueur aléatoire commence

        while not done:
            if turn == 1:  # Tour de l'agent IA
                available_moves = env.available_moves()
                action = ai_agent.choose_action(state, available_moves)
                env.make_move(action, ai_agent.player)
                next_state = env.get_state()

                # Vérifie le résultat après le coup de l'IA
                if env.current_winner == ai_agent.player:
                    ai_agent.update_q_table(state, action, reward=1, next_state=next_state, done=True)
                    win += 1
                    done = True
                elif env.is_draw():
                    ai_agent.update_q_table(state, action, reward=0.5, next_state=next_state, done=True)
                    draw += 1
                    done = True
                else:
                    ai_agent.update_q_table(state, action, reward=0, next_state=next_state, done=False)

                state = next_state
            else:  # Tour du joueur aléatoire
                available_moves = env.available_moves()
                action = random_player_move(available_moves)
                env.make_move(action, -ai_agent.player)
                next_state = env.get_state()

                # Vérifie le résultat après le coup du joueur aléatoire
                if env.current_winner == -ai_agent.player:
                    ai_agent.update_q_table(state, action, reward=-1, next_state=next_state, done=True)
                    loss += 1
                    done = True
                elif env.is_draw():
                    ai_agent.update_q_table(state, action, reward=0.5, next_state=next_state, done=True)
                    draw += 1
                    done = True
                else:
                    # Pas de mise à jour pour le joueur aléatoire
                    state = next_state

            turn *= -1  # Change le tour

        # Décroît l'exploration de l'IA après chaque partie
        ai_agent.decay_epsilon()

        # Affiche les statistiques tous les 1000 épisodes pour voir l'avancé de l'IA
        if episode % 1000 == 0:
            print(f"Épisode {episode} - Victoires de l'IA: {win}, Défaites de l'IA: {loss}, Égalités: {draw}")
            win, loss, draw = 0, 0, 0  # Réinitialise les compteurs pour la prochaine série

    print("Entraînement terminé")

In [4]:
def trainAiVsMinMaxer(agent, minimax_agent, env, episodes=10000, print_every=1000, player=True):
    win_agent, win_minimax, draw = 0, 0, 0  # Compteurs pour les statistiques

    for episode in range(1, episodes + 1):
        board = env.reset()  # Réinitialise l'environnement pour chaque partie
        done = False
        turn = 1 if player else -1  # Si l'agent Q-learning commence (joueur 1), sinon l'agent Minimax

        while not done:
            if turn == 1:  # Tour de l'agent Q-learning
                available_actions = env.available_moves()
                action = agent.choose_action(board, available_actions)
                if env.make_move(action, 1):  # L'agent Q-learning (joueur 1) fait son mouvement
                    if env.current_winner == 1:  # Vérifier si l'agent Q-learning a gagné
                        agent.update_q_table(board, action, reward=1, next_state=env.get_state(), done=True)
                        win_agent += 1
                        done = True
                    elif env.is_draw():  # Vérifier si c'est une égalité
                        agent.update_q_table(board, action, reward=0.5, next_state=env.get_state(), done=True)
                        draw += 1
                        done = True
                    else:
                        agent.update_q_table(board, action, reward=0, next_state=env.get_state(), done=False)
                board = env.get_state()

            else:  # Tour de l'agent Minimax
                available_actions = env.available_moves()
                action = minimax_agent.choose_action(board, env)  # Choisir le coup avec Minimax
                if env.make_move(action, -1):  # L'agent Minimax (joueur -1) fait son mouvement
                    if env.current_winner == -1:  # Vérifier si l'agent Minimax a gagné
                        agent.update_q_table(board, action, reward=-2, next_state=env.get_state(), done=True)
                        win_minimax += 1
                        done = True
                    elif env.is_draw():  # Vérifier si c'est une égalité
                        agent.update_q_table(board, action, reward=0.5, next_state=env.get_state(), done=True)
                        draw += 1
                        done = True
                board = env.get_state()

            # Vérification de l'égalité
            if env.is_draw() and not done:
                agent.update_q_table(board, action, reward=0.5, next_state=env.get_state(), done=True)
                draw += 1
                done = True

            turn *= -1  # Change le tour

        # Décroît l'exploration de l'agent Q-learning après chaque partie
        agent.decay_epsilon()

        # Affiche les statistiques tous les `print_every` épisodes
        if episode % print_every == 0:
            print(f"Épisode {episode} - Victoires Agent: {win_agent}, Victoires Minimax: {win_minimax}, Égalités: {draw}")
            win_agent, win_minimax, draw = 0, 0, 0  # Réinitialise les compteurs pour la prochaine série

    # Affiche les statistiques finales après l'entraînement complet
    print("Entraînement terminé")


In [2]:
import random

def trainAiVsPreProgrammedAgent(q_agent, preprogrammed_agent, env, episodes=10000, print_every=1000, q_agent_first=True):
    win_q_agent, win_preprogrammed, draw = 0, 0, 0  # Counters for stats

    for episode in range(1, episodes + 1):
        board = env.reset()  # Reset the environment at the start of each episode
        done = False
        turn = 1 if q_agent_first else -1  # Q-learning agent goes first if q_agent_first is True

        while not done:
            if turn == 1:  # Q-learning agent's turn
                available_moves = env.available_moves()
                action = q_agent.choose_action(board, available_moves)
                if env.make_move(action, q_agent.player):
                    next_state = env.get_state()
                    if env.current_winner == q_agent.player:  # Q-learning agent wins
                        q_agent.update_q_table(board, action, reward=1, next_state=next_state, done=True)
                        win_q_agent += 1
                        done = True
                    elif env.is_draw():  # Draw game
                        q_agent.update_q_table(board, action, reward=0.5, next_state=next_state, done=True)
                        draw += 1
                        done = True
                    else:
                        q_agent.update_q_table(board, action, reward=0, next_state=next_state, done=False)
                board = next_state

            else:  # PreProgrammedAgent's turn
                action = preprogrammed_agent.choose_action(board, env)
                if env.make_move(action, preprogrammed_agent.player):
                    if env.current_winner == preprogrammed_agent.player:  # PreProgrammedAgent wins
                        q_agent.update_q_table(board, action, reward=-1, next_state=env.get_state(), done=True)
                        win_preprogrammed += 1
                        done = True
                    elif env.is_draw():  # Draw game
                        q_agent.update_q_table(board, action, reward=0.5, next_state=env.get_state(), done=True)
                        draw += 1
                        done = True
                board = env.get_state()

            turn *= -1  # Switch turns

        # Decay exploration rate for the Q-learning agent after each episode
        q_agent.decay_epsilon()

        # Print stats every `print_every` episodes
        if episode % print_every == 0:
            print(f"Episode {episode} - Q-learning Wins: {win_q_agent}, PreProgrammed Wins: {win_preprogrammed}, Draws: {draw}")
            win_q_agent, win_preprogrammed, draw = 0, 0, 0  # Reset counters for the next print interval

    print("Training Complete")

In [5]:
import random

def trainAivsAi(agent1, agent2, env, episodes=10000, print_every=1000):
  
    win_agent1, win_agent2, draw = 0, 0, 0  # Compteurs pour les statistiques

    for episode in range(1, episodes + 1):
        state = env.reset()  # Réinitialise l'environnement pour chaque partie
        done = False
        turn = 1  # 1 pour agent1, -1 pour agent2

        while not done:
            if turn == 1:  # Tour de l'agent1
                available_moves = env.available_moves()
                action = agent1.choose_action(state, available_moves)
                env.make_move(action, agent1.player)
                next_state = env.get_state()

                # Vérifie le résultat après le coup de l'agent1
                if env.current_winner == agent1.player:
                    agent1.update_q_table(state, action, reward=1, next_state=next_state, done=True)
                    agent2.update_q_table(state, action, reward=-1, next_state=next_state, done=True)
                    win_agent1 += 1
                    done = True
                elif env.is_draw():
                    agent1.update_q_table(state, action, reward=1, next_state=next_state, done=True)
                    agent2.update_q_table(state, action, reward=1.5, next_state=next_state, done=True)
                    draw += 1
                    done = True
                else:
                    agent1.update_q_table(state, action, reward=0, next_state=next_state, done=False)
                state = next_state
            else:  # Tour de l'agent2
                available_moves = env.available_moves()
                action = agent2.choose_action(state, available_moves)
                env.make_move(action, agent2.player)
                next_state = env.get_state()

                # Vérifie le résultat après le coup de l'agent2
                if env.current_winner == agent2.player:
                    agent1.update_q_table(state, action, reward=-2, next_state=next_state, done=True)
                    agent2.update_q_table(state, action, reward=2, next_state=next_state, done=True)
                    win_agent2 += 1
                    done = True
                elif env.is_draw():
                    agent1.update_q_table(state, action, reward=1, next_state=next_state, done=True)
                    agent2.update_q_table(state, action, reward=1.5, next_state=next_state, done=True)
                    draw += 1
                    done = True
                else:
                    agent2.update_q_table(state, action, reward=0, next_state=next_state, done=False)
                state = next_state

            turn *= -1  # Change le tour

        # Décroît l'exploration de chaque IA après chaque partie
        agent1.decay_epsilon()
        agent2.decay_epsilon()

        # Affiche les statistiques tous les `print_every` épisodes
        if episode % print_every == 0:
            print(f"Épisode {episode} - Victoires Agent1: {win_agent1}, Victoires Agent2: {win_agent2}, Égalités: {draw}")
            win_agent1, win_agent2, draw = 0, 0, 0  # Réinitialise les compteurs pour la prochaine série

    # Affiche les statistiques finales après l'entraînement complet
    print("Entraînement terminé")


In [6]:
class MinimaxAgent:
    def __init__(self, player):
        self.player = player
        self.transposition_table = {}  # Initialize the transposition table

    def choose_action(self, board, env):
        """
        Choisir le meilleur coup en utilisant l'algorithme Minimax avec élagage alpha-bêta.
        """
        best_move = None
        best_score = -float('inf') if self.player == 1 else float('inf')
        alpha = -float('inf')
        beta = float('inf')

        for move in env.available_moves():
            new_board = list(board) 
            new_board[move] = self.player
            score = self.minimax(new_board, env, -self.player, alpha, beta)  # Switch player for minimax

            if (self.player == 1 and score > best_score) or (self.player == -1 and score < best_score):
                best_score = score
                best_move = move

            # Update alpha or beta based on the current player
            if self.player == 1:
                alpha = max(alpha, best_score)
            else:
                beta = min(beta, best_score)

        return best_move
    
    def minimax(self, board, env, player, alpha, beta):
        """
        Implémentation de l'algorithme Minimax avec élagage alpha-bêta et table de transposition.
        """
        board_hash = str(board)  # Create a hashable representation of the board

        # Check if the board state is already in the transposition table
        if board_hash in self.transposition_table:
            entry = self.transposition_table[board_hash]
            if entry[0] >= beta:  # If the stored score is greater than or equal to beta
                return entry[0]  # Return the stored score (beta cutoff)
            if entry[1] <= alpha:  # If the stored score is less than or equal to alpha
                return entry[1]  # Return the stored score (alpha cutoff)
            alpha = max(alpha, entry[0])  # Update alpha if necessary
            beta = min(beta, entry[1])  # Update beta if necessary

        if self.check_winner(board, 1):  
            return 1
        if self.check_winner(board, -1):  
            return -1
        if env.is_draw():  
            return 0

        best_score = -float('inf') if player == 1 else float('inf')
        for move in env.available_moves():
            env.make_move(move, player)
            score = self.minimax(env.board, env, -player, alpha, beta)
            env.board[move] = 0

            if player == 1:
                best_score = max(best_score, score)
                alpha = max(alpha, best_score)
                if alpha >= beta:
                    break  # Beta cutoff
            else:
                best_score = min(best_score, score)
                beta = min(beta, best_score)
                if beta <= alpha:
                    break  # Alpha cutoff
                    
        # Store the result in the transposition table
        if best_score <= alpha:
            self.transposition_table[board_hash] = (best_score, best_score)  # Store as (score, score) for simplicity
        elif best_score >= beta:
            self.transposition_table[board_hash] = (best_score, best_score)
        else:
            self.transposition_table[board_hash] = (alpha, beta)  # Store the updated alpha and beta values

        return best_score

    

    def check_winner(self, board, player):  
        win_conditions = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # lignes
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # colonnes
            [0, 4, 8], [2, 4, 6]  # diagonales
        ]
        for condition in win_conditions:
            if all(board[i] == player for i in condition):
                return True
        return False

In [1]:
import random

class PreProgrammedAgent:
    def __init__(self, player):
        self.player = player  # 1 for X, -1 for O
        self.opponent = -player  # Opposite player

    def choose_action(self, board, env):
        available_moves = env.available_moves()
        
        # Check if there's a winning move for the agent
        for move in available_moves:
            new_board = list(board)
            new_board[move] = self.player
            if env.check_winner(move, self.player):
                return move  # Play the winning move

        # Check if the opponent has a winning move to block
        for move in available_moves:
            new_board = list(board)
            new_board[move] = self.opponent
            if env.check_winner(move, self.opponent):
                return move  # Block the opponent

        # Look for a move that could lead to a win in the future
        for move in available_moves:
            new_board = list(board)
            new_board[move] = self.player
            if any(env.check_winner(future_move, self.player) for future_move in env.available_moves()):
                return move

        # If no strategic moves, play a random move
        return random.choice(available_moves)

In [7]:

class QLearningAgent:
    def __init__(self, player, alpha=0.1, gamma=0.9, epsilon=1.0, epsilon_decay=0.99):
        self.q_table = {}  # Dictionnaire pour stocker Q-values
        self.alpha = alpha  # Taux d'apprentissage
        self.gamma = gamma  # Facteur de réduction
        self.epsilon = epsilon  # Facteur d'exploration
        self.epsilon_decay = epsilon_decay  # Décroissance de l'exploration
        self.player = player

    def get_q_value(self, state, action):
        # Récupère la Q-value pour une action donnée dans un état donné
        return self.q_table.get((state, action), 0.0)

    
    def choose_action(self, state, available_actions):
        # Choisir une action selon la politique ε-greedy
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(available_actions)  # Exploration
        else:
            # Exploitation: choisir l'action avec la plus grande Q-value
            q_values = [self.get_q_value(state, action) for action in available_actions]
            max_q_value = max(q_values)
            max_q_actions = [action for action in available_actions if self.get_q_value(state, action) == max_q_value]
            return random.choice(max_q_actions)

    def update_q_table(self, state, action, reward, next_state, done):
        # Mise à jour de la Q-value
        max_future_q = max([self.get_q_value(next_state, a) for a in range(9)], default=0)
        current_q = self.get_q_value(state, action)
        if done:
            new_q = reward
        else:
            new_q = current_q + self.alpha * (reward + self.gamma * max_future_q - current_q)
        self.q_table[(state, action)] = new_q

    def decay_epsilon(self):
        self.epsilon *= self.epsilon_decay


On créer un environnement de jeu qui sera utilisé pendant tout le test

In [8]:
env = TicTacToeEnv()

On initialise 2 agents qui vont s'entrainer respectivement a chaque role (1er joueur/ 2eme joueur

In [9]:
agent_First = QLearningAgent(player=1)
agent_Second = QLearningAgent(player=-1)
agent_MinMaxer_1 = MinimaxAgent(player=1)
agent_MinMaxer_2 = MinimaxAgent(player=-1)

In [ ]:
trainAiVsSmartAgent(agent_First,agent_MinMaxer_2, env, 10000, True)

Épisode 1 - Victoires Agent: 1, Victoires Minimax: 0, Égalités: 0
Épisode 2 - Victoires Agent: 1, Victoires Minimax: 0, Égalités: 0
Épisode 3 - Victoires Agent: 1, Victoires Minimax: 0, Égalités: 0
Épisode 4 - Victoires Agent: 1, Victoires Minimax: 0, Égalités: 0
Épisode 5 - Victoires Agent: 1, Victoires Minimax: 0, Égalités: 0
Épisode 6 - Victoires Agent: 1, Victoires Minimax: 0, Égalités: 0
Épisode 7 - Victoires Agent: 1, Victoires Minimax: 0, Égalités: 0
Épisode 8 - Victoires Agent: 1, Victoires Minimax: 0, Égalités: 0
Épisode 9 - Victoires Agent: 1, Victoires Minimax: 0, Égalités: 0
Épisode 10 - Victoires Agent: 1, Victoires Minimax: 0, Égalités: 0
Épisode 11 - Victoires Agent: 1, Victoires Minimax: 0, Égalités: 0
Épisode 12 - Victoires Agent: 0, Victoires Minimax: 1, Égalités: 0
Épisode 13 - Victoires Agent: 1, Victoires Minimax: 0, Égalités: 0
Épisode 14 - Victoires Agent: 1, Victoires Minimax: 0, Égalités: 0
Épisode 15 - Victoires Agent: 0, Victoires Minimax: 1, Égalités: 0
Épis

In [1]:
trainAiVsRandom(agent_First, env, 10000, False)

NameError: name 'trainAiVsRandom' is not defined

In [21]:
trainAiVsRandom(agent_Second, env, 10000, False)

Épisode 1000 - Victoires de l'IA: 519, Défaites de l'IA: 336, Égalités: 145
Épisode 2000 - Victoires de l'IA: 482, Défaites de l'IA: 363, Égalités: 155
Épisode 3000 - Victoires de l'IA: 530, Défaites de l'IA: 322, Égalités: 148
Épisode 4000 - Victoires de l'IA: 516, Défaites de l'IA: 311, Égalités: 173
Épisode 5000 - Victoires de l'IA: 523, Défaites de l'IA: 331, Égalités: 146
Épisode 6000 - Victoires de l'IA: 511, Défaites de l'IA: 331, Égalités: 158
Épisode 7000 - Victoires de l'IA: 524, Défaites de l'IA: 326, Égalités: 150
Épisode 8000 - Victoires de l'IA: 507, Défaites de l'IA: 333, Égalités: 160
Épisode 9000 - Victoires de l'IA: 475, Défaites de l'IA: 364, Égalités: 161
Épisode 10000 - Victoires de l'IA: 530, Défaites de l'IA: 316, Égalités: 154
Entraînement terminé


In [25]:
trainAivsAi(agent_First, agent_Second, env)

# Afficher le Q-table
#print(agent_First.q_table)
#print(agent_Second.q_table)

Épisode 1000 - Victoires Agent1: 611, Victoires Agent2: 300, Égalités: 89
Épisode 2000 - Victoires Agent1: 619, Victoires Agent2: 297, Égalités: 84
Épisode 3000 - Victoires Agent1: 638, Victoires Agent2: 275, Égalités: 87
Épisode 4000 - Victoires Agent1: 656, Victoires Agent2: 256, Égalités: 88
Épisode 5000 - Victoires Agent1: 640, Victoires Agent2: 277, Égalités: 83
Épisode 6000 - Victoires Agent1: 635, Victoires Agent2: 276, Égalités: 89
Épisode 7000 - Victoires Agent1: 653, Victoires Agent2: 275, Égalités: 72
Épisode 8000 - Victoires Agent1: 634, Victoires Agent2: 289, Égalités: 77
Épisode 9000 - Victoires Agent1: 662, Victoires Agent2: 257, Égalités: 81
Épisode 10000 - Victoires Agent1: 648, Victoires Agent2: 269, Égalités: 83
Entraînement terminé


In [15]:
trainAiVsRandom(agent_First, env, 10000, True)

Épisode 1000 - Victoires de l'IA: 765, Défaites de l'IA: 145, Égalités: 90
Épisode 2000 - Victoires de l'IA: 785, Défaites de l'IA: 150, Égalités: 65
Épisode 3000 - Victoires de l'IA: 766, Défaites de l'IA: 155, Égalités: 79
Épisode 4000 - Victoires de l'IA: 797, Défaites de l'IA: 142, Égalités: 61
Épisode 5000 - Victoires de l'IA: 791, Défaites de l'IA: 137, Égalités: 72
Épisode 6000 - Victoires de l'IA: 796, Défaites de l'IA: 136, Égalités: 68
Épisode 7000 - Victoires de l'IA: 795, Défaites de l'IA: 136, Égalités: 69
Épisode 8000 - Victoires de l'IA: 783, Défaites de l'IA: 141, Égalités: 76
Épisode 9000 - Victoires de l'IA: 781, Défaites de l'IA: 139, Égalités: 80
Épisode 10000 - Victoires de l'IA: 800, Défaites de l'IA: 121, Égalités: 79
Entraînement terminé


In [16]:
trainAiVsRandom(agent_Second, env, 10000, False)


Épisode 1000 - Victoires de l'IA: 526, Défaites de l'IA: 296, Égalités: 178
Épisode 2000 - Victoires de l'IA: 524, Défaites de l'IA: 333, Égalités: 143
Épisode 3000 - Victoires de l'IA: 498, Défaites de l'IA: 331, Égalités: 171
Épisode 4000 - Victoires de l'IA: 510, Défaites de l'IA: 319, Égalités: 171
Épisode 5000 - Victoires de l'IA: 513, Défaites de l'IA: 325, Égalités: 162
Épisode 6000 - Victoires de l'IA: 499, Défaites de l'IA: 331, Égalités: 170
Épisode 7000 - Victoires de l'IA: 494, Défaites de l'IA: 336, Égalités: 170
Épisode 8000 - Victoires de l'IA: 529, Défaites de l'IA: 326, Égalités: 145
Épisode 9000 - Victoires de l'IA: 513, Défaites de l'IA: 334, Égalités: 153
Épisode 10000 - Victoires de l'IA: 504, Défaites de l'IA: 325, Égalités: 171
Entraînement terminé
